In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import wav2vec2, bert






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
class Encoder:
    def __init__(self, config):
        self.config = config
        self.model = None
    def __call__(self, x, input_mask, training = True):
        if self.model is None:
            input_mask = tf.logical_not(input_mask)
            self.model = bert.BertModel(config = self.config, is_training = training,
                                  input_ids = x, input_mask = input_mask)
        return self.model.sequence_output

In [6]:
encoder = Encoder(config = bert.BertConfig())

In [7]:
cfg = wav2vec2.Wav2Vec2Config()
model = wav2vec2.Model(cfg, encoder)

In [8]:
X = tf.placeholder(tf.float32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))

In [9]:
r = model(X, padding_mask = X_len)
r

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
dim is deprecated, use axis instead


{'x': <tf.Tensor 'wav2vec2/truediv_7:0' shape=(101, ?, ?) dtype=float32>,
 'padding_mask': <tf.Tensor 'wav2vec2/LogicalNot:0' shape=(?, ?) dtype=bool>,
 'features_pen': <tf.Tensor 'wav2vec2/Mean:0' shape=() dtype=float32>}

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

3.518625

In [16]:
y = np.random.normal(size = (16000 * 5))

In [17]:
o = sess.run(r, feed_dict = {X: [y, y], X_len: [len(y), len(y)]})

In [18]:
o['x'].shape

(101, 2, 249)

In [23]:
# import torch
# x = torch.from_numpy(np.array([[1,1,2,2]]))
# x.new_zeros(x.size(0) * x.size(1))